<a href="https://colab.research.google.com/github/Alona-Aldushyna/Alona-Aldushyna-s-the-Capstone-Project/blob/main/proposed_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.15.0
Hub version: 0.16.1
GPU is available


In [29]:
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D

In [30]:
from keras.preprocessing import image
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/validation')

math_formulas_types = os.listdir('/content/drive/MyDrive/validation')
print (math_formulas_types)  #what kinds of rooms are in this dataset

print("Types of math formulas found: ", len(dataset_path))

['8_jpg', '9_jpg', '4_jpg', '+_jpg', '-_jpg', '6_jpg', '=_jpg', '7_jpg', 'dev_jpg', '5_jpg', '1_jpg', '2_jpg', '3_jpg', '0_jpg']
Types of math formulas found:  14


In [32]:
formulas = []

for item in math_formulas_types:
 # Get all the file names
 all_formulas = os.listdir('/content/drive/MyDrive/validation' + '/' +item)
 #print(all_shoes)

 # Add them to the list
 for formula in  all_formulas:
    formulas.append((item, str('/content/drive/MyDrive/validation' + '/' +item) + '/' + formula))
    print(formulas)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_6162.jpg'), ('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_4437.jpg'), ('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_9811.jpg'), ('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_899.jpg'), ('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_2518.jpg'), ('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_2589.jpg'), ('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_5825.jpg'), ('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_6642.jpg'), ('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_3632.jpg'), ('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_7295.jpg'), ('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_6243.jpg'), ('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_6001.jpg'), ('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_2809.jpg'), ('8_jpg', '/content/drive/MyDrive/validation/8_jpg/aug_0_2747.jpg'), ('8_jpg', '/content/drive/MyDrive/

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
# Build a dataframe
formulas_df = pd.DataFrame(data=formulas, columns=['formulas type', 'image'])
print(formulas_df.head())

  formulas type                                              image
0         8_jpg  /content/drive/MyDrive/validation/8_jpg/aug_0_...
1         8_jpg  /content/drive/MyDrive/validation/8_jpg/aug_0_...
2         8_jpg  /content/drive/MyDrive/validation/8_jpg/aug_0_...
3         8_jpg  /content/drive/MyDrive/validation/8_jpg/aug_0_...
4         8_jpg  /content/drive/MyDrive/validation/8_jpg/aug_0_...


In [34]:
import cv2
path = '/content/drive/MyDrive/validation/'


im_size = 32

images = []
labels = []

for i in math_formulas_types:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path) ]

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [35]:
images = np.array(images)

images = images.astype('float32') / 255.0
images.shape

(9156, 32, 32, 3)

In [36]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Предположим, что y - это одномерный массив меток классов (3 класса)
y = formulas_df['formulas type'].values

# Используем LabelEncoder для преобразования строковых меток в числовой формат
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Используем OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
Y_onehot = onehot_encoder.fit_transform(y_encoded.reshape(-1, 1))

print(Y_onehot)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [37]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y_onehot = shuffle(images, Y_onehot, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y_onehot, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(8698, 32, 32, 3)
(8698, 14)
(458, 32, 32, 3)
(458, 14)


In [49]:
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.regularizers import l2

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001), input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))  # Добавление dropout для регуляризации
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dense(14))  # Выходной слой с 14 нейронами для классификации ваших 14 классов


In [50]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 15, 15, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 6, 6, 64)          0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 2, 2, 128)        

In [51]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_x, train_y, epochs=40, validation_data=(test_x, test_y))

Epoch 1/40
272/272 [==============================] - 3s 6ms/step - loss: 8.4976 - accuracy: 0.0658 - val_loss: 8.3262 - val_accuracy: 0.0786
Epoch 2/40
272/272 [==============================] - 1s 5ms/step - loss: 8.6395 - accuracy: 0.0700 - val_loss: 9.2714 - val_accuracy: 0.1026
Epoch 3/40
272/272 [==============================] - 1s 5ms/step - loss: 8.4405 - accuracy: 0.0722 - val_loss: 7.9685 - val_accuracy: 0.0830
Epoch 4/40
272/272 [==============================] - 1s 4ms/step - loss: 8.4871 - accuracy: 0.0690 - val_loss: 7.9228 - val_accuracy: 0.0786
Epoch 5/40
272/272 [==============================] - 1s 5ms/step - loss: 8.4954 - accuracy: 0.0722 - val_loss: 8.8971 - val_accuracy: 0.0786
Epoch 6/40
272/272 [==============================] - 1s 5ms/step - loss: 8.4071 - accuracy: 0.0739 - val_loss: 8.8884 - val_accuracy: 0.0786
Epoch 7/40
272/272 [==============================] - 1s 5ms/step - loss: 8.3429 - accuracy: 0.0711 - val_loss: 8.8810 - val_accuracy: 0.0939
Epoch 

KeyboardInterrupt: 